## Context Managers in Python
Context managers allow us to manage the contexts (ressources and scope) of our block of operations. They are implmented with `with` statement.

**The `with` statement** 
The with statement is used to wrap the execution of a block with methods defined by a context manager.
The execution of with staement with one 'item' proceeds as follow:
1. The context expression is evaluated to obtain the context manager.
2. The context manager's `__enter__()` is loaded for later use.
3. The context manager's `__exit__()` is loaded for later use
4. The context manager's `__enter__()` is invoked.
5. If a target was included in the with statement, the return value from `__enter__()` is assigned to it.
6. The suite is executed.
7. The context managers `__exit__()` method is invoked. If exception occurs, its type, value and traceback are passed as arguments to `__exit__()`, otherwise `None` arguments are supplied.

If the suite was exited due to an exception, and return value from the `__exit__()` was false, the exception is reraised.

If the return value was true, the exception is suppressed, and execution with the statement after `with` continues.

If the exit was due to any other reason than exception, the return value from `__exit__()` is ignored, and execution proceeds at the normal location for the kind of exit.

**Simple example of context managers is `with` `open` operation.**

In [1]:
with open('some_file', 'w') as file:
    file.write('Hola como estas!')

The above code is equivalent to :


In [3]:
file = open('some_file_1','w')
try:
    file.write('Hoa comoa trestas!')
finally:
    file.close()

Compared to the first example, the second example has unnecessary boilerplate that we don't require. Andother advantage of `with` statement is that it makes sure our file is closed outside of our scope. A common use case of context manager is locking and unlocking the ressources and closing/opening files.

**Now let's implment our own context manager** 

### Implementing a context manager as a Class
If we are implementing it, our CM needs `__enter__` and `__exit` methods defined at least. Let's implement our own CM for the file handling

In [4]:
# File handling context manager
class HandleFile(object):
    '''Handles the file operation as CM'''
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, traceback):
        self.file_obj.close()

In [6]:
# Let's use our context manager
with HandleFile('demo.txt','w') as opened_file:
    opened_file.write('Arigato!')

### Handling Exceptions
Between 4th and 5th step if the error occurs, the `__exit__` method decides what to do with errors and exits the suite.

In [7]:
with HandleFile('demo','w') as opened_file:
    opened_file.function_is_not_defined('LOL')

AttributeError: '_io.TextIOWrapper' object has no attribute 'function_is_not_defined'

Here, In our case `__exit__` method returns `None` therefore with stament raises the exceptions. Now let's try handling the exception in `__exit__` method


In [8]:
class Filer(object):
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
    def __enter__(self):
        return self.file_obj
    def __exit__(self, type, value, traceback):
        print(f"Exception was:\n Type: {str(type)} \nValue: {str(value)}\nTraceback: {str(traceback)}")

In [9]:
# Now let's see what happens if the error occurs
with Filer('demo','w') as opened_file:
    opened_file.function_is_not_defined('LOL')

Exception was:
 Type: <class 'AttributeError'> 
Value: '_io.TextIOWrapper' object has no attribute 'function_is_not_defined'
Traceback: <traceback object at 0x7fdebb2f3c80>


AttributeError: '_io.TextIOWrapper' object has no attribute 'function_is_not_defined'

**As we can see, we can access all of the details about our Exception and handle them gracefully**

### Implementing context manager as a generator
Context Managers can also be implemented using decorators and generators. Python has `contextlib` module for this very purpose. So, instead of a class -- we implment our CMs using a generator function.

In [11]:
# Basic context manger with yield and contxtlib
from contextlib import contextmanager
@contextmanager
def open_file(name):
    f = open(name,'w')
    try:
        yield f
    finally:
        f.close()


Here,
1. Python sees a `yield` keyword. Due to this it creates a generator instead of formal function.
2. Due to the decoration context manger is called with the function name `open_file` as its argument.
3. `contextmanager` decorator returns the generator wrapped by the `GeneratorContextManager` object
4. The generator object is assigned to the open_file function, there when we later call the open_file function, we are calling the `GeneratorContextManagers` object.

In [12]:
# using generator context managers
with open_file('some_stuff') as f:
    f.write("Something something")

## Async Context Managers
Async context manager is a context managers that is able to suspend execution in its `__aenter__` and `__aexit__` methods. They can be used with `async with` statement.

The statement `aenter` and `aexit` are equivalent to `enter` and `exit` dunder methods

**`async with`**
```python
async with EXPRESSION as TARGET:
    SUITE
```
and is symantically equivalent to:
```python
manager = (EXPRESSION)
aenter = type(manager).__aenter__
aexit = type(manager).__aexit__
value = await aenter(manager)
hit_except = False

try:
    TARGET = value
    SUITE
except:
    hit_except = True
    if not await aexit(manager, *sys.exc_info()):
        raise
finally:
    if not hit_except:
        await aexit(manager, None, None, None)
```

In [25]:
# simple example of async context manager
import time


async def slow_msg(message: str, times: int):
    for i in range(times):
        print(message)
        time.sleep(1)

        
async def log(message: str):
    time.sleep(0.5)
    print(message)
    
    
class AsyncContextManager:
    def __init__(self, frequency: int = 3, msg: str = "Hello from async CM."):
        self.frequency = frequency
        self.msg = msg
    async def __aenter__(self):
        await log('Entering the context')
        await slow_msg(self.msg, self.frequency)
    async def __aexit__(self, exc_type, exc, tb):
        await log("Exiting context")

**Here we have implemented a simple method that prints our message some number of times with 1 second gap, and we are using our AsyncContextManager to call and manage it**

In [26]:
async with AsyncContextManager(5,"Hello World!") as t:
    pass
     

Entering the context
Hello World!
Hello World!
Hello World!
Hello World!
Hello World!
Exiting context


## Implementing async context managers with generators
Similar to context managers, async context managers can be implemented with the decorator and generators. Python provides `asynccontextmanager` decorator for the job.

This function is a decorator that can be used to define a factory function for async with statement async context managers, without needing to recreate class or separate async dunders.

In [31]:
# simple async context manager with decorator
from contextlib import asynccontextmanager
import time

async def acquire_db_connection():
    print("Acquiring db connection...")
    time.sleep(1)
    print("Connection Acquired")

async def release_db_connection(conn):
    print("Closing db")
    time.sleep(1)
    print("Connection closed")
    
@asynccontextmanager
async def get_connection():
    await acquire_db_connection()
    try:
        yield 5
    finally:
        await release_db_connection()

In [ ]:
async with get_connection() as conn:
    print("Success")

Acquiring db connection...
